<a href="https://colab.research.google.com/github/babypotatotang/Introduction-to-DeepLearning/blob/main/6.%20Artifical%20Neural%20Network/08.%20%EB%8B%A4%EC%B8%B5%20%ED%8D%BC%EC%85%89%ED%8A%B8%EB%A1%A0%EC%9C%BC%EB%A1%9C%20MNIST%EB%B6%84%EB%A5%98%ED%95%98%EA%B8%B0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#1. 데이터 로드하기
---

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import fetch_openml # 데이터셋 load

In [ ]:
mnist = fetch_openml('mnist_784', version=1, cache=True, as_frame = False)

In [ ]:
mnist.data

In [ ]:
mnist.target[0]

In [ ]:
mnist.target = mnist.target.astype(np.int8)

In [ ]:
X = mnist.data / 255  # 0-255값을 [0,1] 구간으로 정규화
y = mnist.target

In [ ]:
X[0]

In [ ]:
y[0]

In [ ]:
plt.imshow(X[0].reshape(28,28), cmap = 'gray')
print(f'이 이미지 데이터의 레이블은 {y[0]}입니다.')

#2. 훈련 데이터와 테스트 데이터의 분리 
---

In [ ]:
import torch 
from torch.utils.data import TensorDataset, DataLoader 
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/7, random_state=0)

X_train = torch.Tensor(X_train)
X_test = torch.Tensor(X_test)
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

ds_train = TensorDataset(X_train, y_train)
ds_test = TensorDataset(X_test, y_test)

loader_train = DataLoader(ds_train, batch_size=64, shuffle=True)
loader_test = DataLoader(ds_test, batch_size=64, shuffle=False)

#3. 다층 퍼셉트론 
---

In [ ]:
from torch import nn 

model = nn.Sequential()
model.add_module('fc1', nn.Linear(28*28*1, 100))
model.add_module('relu1', nn.ReLU())
model.add_module('fc2', nn.Linear(100,100))
model.add_module('relu2', nn.ReLU())
model.add_module('fc3', nn.Linear(100,10))

print(model)

In [ ]:
from torch import optim 

# 오차함수 선택 
loss_fn = nn.CrossEntropyLoss()

# 가중치를 학습하기 위한 최적화 기법 선택 
optimizer = optim.Adam(model.parameters(), lr = 0.01)

In [ ]:
def train(epoch):
    model.train() # 신경마을 학습모드로 전환 

    # 데이터로더에서 미니배치를 하나씩 꺼내 학습을 수행 
    for data, targets in loader_train:
        optimizer.zero_grad() # 경사를 0으로 초기화 

        outputs = model(data) # 데이터를 입력하고 출력을 계산 
        loss = loss_fn(outputs, targets) # 출력과 훈련데이터 정답 간의 오차를 계산 
        
        loss.backward() # 오차를 역전파 계산 
        optimizer.step() # 역전파 계산한 값으로 가중치 수정 

    print(f"epoch {epoch} 완료")

In [ ]:
def test():
    model.eval() # 신경망을 추론모드로 전환
    correct = 0 

    # 데이터로더에서 미니배치를 하나씩 꺼내 추론을 수행 
    with torch.no_grad(): # 추론 과정에는 미분이 필요 없음. 
        for data, targets in loader_test:
            outputs = model(data) # 데이터를 입력하고 출력 계산 

            # 추론 계산 
            _, predicted = torch.max(outputs.data, 1) # 확률이 가장 높은 레이블이 무엇인지 계산 
            correct += predicted.eq(targets.data.view_as(predicted)).sum() # 정답과 일치한 경우 정답 카운트 증가 

            # 정확도 출력 
            data_num = len(loader_test.dataset) # 데이터 총 건수 

    print(f"테스트 데이터에서 예측 정확도 : {correct} / {data_num} {100. * (correct / data_num):.2f}%")
            

In [ ]:
test()

In [ ]:
for epoch in range(3):
    train(epoch)

test()

In [ ]:
index = 2018 

model.eval() # 신경망을 추론 모드로 전환 
data = X_test[index]
output = model(data) # 데이터를 입력하고 출력을 계산 
_, predicted = torch.max(output.data, 0) # 확률이 가장 높은 레이블이 무엇인지 계산 

print(f'예측 결과 : {predicted}')

X_test_show = (X_test[index]).numpy()
plt.imshow(X_test_show.reshape(28,28), cmap = 'gray')
print(f"이 이미지 데이터의 정답 레이블은 {y_test[index]:.0f}")